In [13]:
from tqdm import tqdm

In [16]:
import requests
from bs4 import BeautifulSoup as bs
import json
API_KEY = "9446fd13127051ca7343c2018204779d8007"
keyword = "covid19"
esearch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term="
#covid19&usehistory=true&webenv=true&retmax=10&api_key=9446fd13127051ca7343c2018204779d8007"
search_resp = requests.get(esearch+keyword+"&usehistory=true&webenv=true"+"api_key="+API_KEY)
search_soup = bs(search_resp.content,features="lxml")
webenv = search_soup.find("webenv").text
#count = search_soup.find("count").text
count =10
batch_size = 2
batches = []


In [18]:
for batch in tqdm(range(0,int(count),2)):
    efetch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&query_key=1&webenv="    
    fetch_resp = requests.get(efetch+webenv+"&rettype=xml&api_key="+API_KEY+"&retmode=xml&retmax=2000&retstart="+str(batch))
    fetch_soup = bs(fetch_resp.content,features="lxml")
    articles_list = []
    PubmedArticleSet = fetch_soup.find("pubmedarticleset")
    articles = PubmedArticleSet.find_all("pubmedarticle")
    for article in tqdm(articles):
        article_dict = {}
        title = article.find("articletitle").text        
        pmid = article.find("pmid").text    
        authors =  article.find("authorlist")
        
        author_list = []
        try:
            author_l = authors.find_all("author")
        except AttributeError:
            break        
        for auth in author_l:
            author_dict = {}
            try:
                first_name = auth.find("forename").text            
            except:
                first_name = ""            
            try:
                second_name = auth.find("lastname").text            
            except:
                second_name = ""           
            try:
                initials = auth.find("initials").text           
            except:
                initials = ""            
            author_name = initials+" " +first_name +" "+second_name            
            try:
                affiliation = auth.find("affiliation").text            
            except:
                affiliation = ""            
            author_dict["author_name"] = author_name            
            author_dict["affiliation"] = affiliation            
            author_list.append(author_dict)
        article_dict["title"] = title        
        article_dict["pmid"] = pmid        
        article_dict["author"] = author_list    
    batches.append(article_dict)
    print(batches)
    print("<------------------------------------------------------------------------------------------------>")
    

 20%|████████████████▊                                                                   | 1/5 [00:45<03:01, 45.49s/it]

[{}, {}, {}]
<------------------------------------------------------------------------------------------------>



 40%|█████████████████████████████████▌                                                  | 2/5 [01:36<02:21, 47.16s/it]

[{}, {}, {}, {}]
<------------------------------------------------------------------------------------------------>



 60%|██████████████████████████████████████████████████▍                                 | 3/5 [02:18<01:31, 45.57s/it]

[{}, {}, {}, {}, {}]
<------------------------------------------------------------------------------------------------>



 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [03:01<00:44, 44.85s/it]

[{}, {}, {}, {}, {}, {}]
<------------------------------------------------------------------------------------------------>



100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:44<00:00, 44.95s/it]

[{}, {}, {}, {}, {}, {}, {}]
<------------------------------------------------------------------------------------------------>


In [15]:
print(batches)

[{}]
